In [1]:

import os

# Check current directory
print(f"Current directory: {os.getcwd()}")

# List files in current directory
print(f"\nFiles here: {os.listdir('.')}")

# Check if our project folder exists
project_path = os.path.expanduser("~/labs_hybrid_optimizer")
if os.path.exists(project_path):
    print(f"\n✓ Project folder found at: {project_path}")
    print(f"  Contents: {os.listdir(project_path)}")
else:
    print(f"\n✗ Project folder not found at: {project_path}")
    
    # Search for it
    home = os.path.expanduser("~")
    print(f"\nSearching in home directory ({home})...")
    for item in os.listdir(home):
        print(f"  {item}")


Current directory: /home/jovyan

Files here: ['.bashrc', '2026-NVIDIA', '.config', '.jupyter', '.npm', '.profile', '.bash_logout', '.ipython', '.ipynb_checkpoints', '.local', 'labs_hybrid_optimizer', '.vim', '.virtual_documents', '.aws', '.conda', '.qbraid_restore_progress.json', 'labs_pipeline.ipynb', '.qbraid_priority_restored', '.ssh', '.viminfo', '.mcp', '.qbraid', '.qbraid_data_ready', '.cache', '.pod_agent', '.claude', 'labs_results_n20.json', '.hotdog']

✓ Project folder found at: /home/jovyan/labs_hybrid_optimizer
  Contents: ['utils.py', 'run_labs.py', 'mts_optimizer.py', 'dcqo_sampler.py', 'labs_hybrid_optimizer', 'tda_analysis.py', 'quality_predictor.py', 'diversity_selector.py', 'pipeline.py']


In [2]:
# Cell 1: Core utilities and setup
import numpy as np
from typing import List, Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

# ============== UTILS ==============
def labs_energy(sequence: np.ndarray) -> int:
    """Compute LABS energy E(s) = sum_k C_k^2"""
    N = len(sequence)
    energy = 0
    for k in range(1, N):
        C_k = sum(sequence[i] * sequence[i + k] for i in range(N - k))
        energy += C_k ** 2
    return energy

def merit_factor(sequence: np.ndarray) -> float:
    """Merit factor F = N^2 / (2 * E(s))"""
    N = len(sequence)
    E = labs_energy(sequence)
    return N ** 2 / (2 * E) if E > 0 else float('inf')

def bitstring_to_sequence(bitstring: str) -> np.ndarray:
    """Convert '0110' to [-1, +1, +1, -1]"""
    return np.array([1 if b == '1' else -1 for b in bitstring])

def sequence_to_bitstring(sequence: np.ndarray) -> str:
    """Convert [-1, +1, +1, -1] to '0110'"""
    return ''.join('1' if s == 1 else '0' for s in sequence)

def local_gradient(sequence: np.ndarray) -> np.ndarray:
    """Compute energy change for each single-bit flip"""
    N = len(sequence)
    current_energy = labs_energy(sequence)
    gradients = np.zeros(N)
    for i in range(N):
        flipped = sequence.copy()
        flipped[i] *= -1
        gradients[i] = labs_energy(flipped) - current_energy
    return gradients

print("Utilities loaded successfully!")

# Quick test
test_seq = np.array([1, -1, 1, -1, 1, -1, 1, -1])
print(f"Test sequence energy: {labs_energy(test_seq)}")
print(f"Test sequence merit factor: {merit_factor(test_seq):.3f}")


Utilities loaded successfully!
Test sequence energy: 140
Test sequence merit factor: 0.229


In [3]:
# Cell 2: DCQO Quantum Sampler (FIXED)
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
import numpy as np
from typing import Dict, List, Tuple

class DCQOSampler:
    """Digitized Counterdiabatic Quantum Optimization for LABS"""
    
    def __init__(self, N: int, num_layers: int = 3):
        self.N = N
        self.num_layers = num_layers
    
    def build_circuit(self, params: Dict[str, float] = None) -> QuantumCircuit:
        """Build the DCQO circuit for LABS"""
        N = self.N
        qr = QuantumRegister(N, 'q')
        cr = ClassicalRegister(N, 'c')
        qc = QuantumCircuit(qr, cr)
        
        # Default parameters
        if params is None:
            params = {}
            for layer in range(self.num_layers):
                t = (layer + 1) / self.num_layers
                params[f'gamma_{layer}'] = 0.4 * t
                params[f'alpha_{layer}'] = 0.6 * (1 - t)
                params[f'beta_{layer}'] = 0.15 * np.sin(np.pi * t)
        
        # Initial superposition
        for i in range(N):
            qc.h(i)
        
        # DCQO layers
        for layer in range(self.num_layers):
            t = (layer + 1) / self.num_layers
            gamma = params.get(f'gamma_{layer}', 0.4 * t)
            alpha = params.get(f'alpha_{layer}', 0.6 * (1 - t))
            beta = params.get(f'beta_{layer}', 0.15 * np.sin(np.pi * t))
            
            # Problem Hamiltonian: ZZ interactions
            for i in range(N - 1):
                qc.rzz(gamma * 0.5, i, i + 1)
            
            # Long-range ZZ for autocorrelation
            for k in range(2, min(N, 4)):
                for i in range(N - k):
                    qc.rzz(gamma * 0.25 / k, i, i + k)
            
            # Counterdiabatic Y rotations
            for i in range(N):
                qc.ry(alpha, i)
            
            # Bias field
            for i in range(N):
                qc.rz(beta * (i - N/2) / N, i)
        
        qc.measure(qr, cr)
        return qc
    
    def sample(self, num_samples: int = 1000) -> List[Tuple[np.ndarray, int]]:
        """Generate DCQO samples"""
        qc = self.build_circuit()
        
        # FIX: Explicitly use MPS method for large circuits
        simulator = AerSimulator(method='matrix_product_state')
        job = simulator.run(qc, shots=num_samples)
        counts = job.result().get_counts()
        
        # Convert to sequences with energies
        samples = []
        for bitstring, count in counts.items():
            seq = bitstring_to_sequence(bitstring[::-1])
            energy = labs_energy(seq)
            samples.extend([(seq, energy)] * count)
        
        samples.sort(key=lambda x: x[1])
        return samples

print("DCQO Sampler loaded!")

# Quick test
sampler = DCQOSampler(N=8, num_layers=2)
qc = sampler.build_circuit()
print(f"Circuit: {qc.num_qubits} qubits, depth {qc.depth()}")


DCQO Sampler loaded!
Circuit: 8 qubits, depth 25


In [4]:
# Cell 3: TDA Basin Analysis
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster

# Try to import ripser, fall back to scipy if not available
try:
    import ripser
    RIPSER_AVAILABLE = True
    print("Ripser available for persistent homology")
except ImportError:
    RIPSER_AVAILABLE = False
    print("Ripser not found, using scipy fallback")

class TDABasinAnalyzer:
    """TDA for identifying solution basins using H0 persistent homology"""
    
    def __init__(self, persistence_threshold: float = 0.1):
        self.persistence_threshold = persistence_threshold
        self.persistence_diagram = None
        self.cluster_labels = None
        self.basin_info = None
    
    def compute_persistence(self, sequences: List[np.ndarray], 
                           energies: List[int] = None) -> Dict:
        """Compute H0 persistent homology in Hamming space"""
        n_samples = len(sequences)
        
        # Compute Hamming distance matrix
        dist_matrix = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(i + 1, n_samples):
                d = np.sum(sequences[i] != sequences[j])
                dist_matrix[i, j] = d
                dist_matrix[j, i] = d
        
        # Energy-weighted distance (optional)
        if energies is not None:
            energy_arr = np.array(energies)
            energy_diff = np.abs(energy_arr[:, None] - energy_arr[None, :])
            max_diff = energy_diff.max() if energy_diff.max() > 0 else 1
            dist_matrix = dist_matrix + 0.1 * (energy_diff / max_diff) * dist_matrix.max()
        
        # Compute persistence
        if RIPSER_AVAILABLE:
            result = ripser.ripser(dist_matrix, maxdim=0, distance_matrix=True)
            self.persistence_diagram = result['dgms'][0]
        else:
            # Fallback: approximate via single-linkage
            self.persistence_diagram = self._approximate_persistence(dist_matrix)
        
        self.basin_info = self._analyze_basins(n_samples)
        
        return {
            'persistence_diagram': self.persistence_diagram,
            'basin_info': self.basin_info,
            'distance_matrix': dist_matrix
        }
    
    def _approximate_persistence(self, dist_matrix: np.ndarray) -> np.ndarray:
        """Approximate H0 persistence using single-linkage clustering"""
        condensed = squareform(dist_matrix)
        Z = linkage(condensed, method='single')
        n = dist_matrix.shape[0]
        births = np.zeros(n - 1)
        deaths = Z[:, 2]
        return np.column_stack([births, deaths])
    
    def _analyze_basins(self, n_samples: int) -> Dict:
        """Analyze persistence diagram for basin structure"""
        if self.persistence_diagram is None or len(self.persistence_diagram) == 0:
            return {'num_basins': 1, 'persistence_values': []}
        
        births = self.persistence_diagram[:, 0]
        deaths = self.persistence_diagram[:, 1]
        persistence = deaths - births
        
        finite_mask = np.isfinite(deaths)
        finite_persistence = persistence[finite_mask]
        
        if len(finite_persistence) == 0:
            return {'num_basins': 1, 'persistence_values': []}
        
        median_pers = np.median(finite_persistence)
        q75, q25 = np.percentile(finite_persistence, [75, 25])
        threshold = median_pers + self.persistence_threshold * (q75 - q25)
        
        long_mask = persistence > threshold
        
        return {
            'num_basins': np.sum(long_mask) + 1,
            'threshold': threshold,
            'persistence_values': persistence,
            'median_persistence': median_pers
        }
    
    def assign_clusters(self, sequences: List[np.ndarray], 
                       dist_matrix: np.ndarray = None,
                       n_clusters: int = None) -> np.ndarray:
        """Assign sequences to clusters"""
        n_samples = len(sequences)
        
        if dist_matrix is None:
            dist_matrix = np.zeros((n_samples, n_samples))
            for i in range(n_samples):
                for j in range(i + 1, n_samples):
                    d = np.sum(sequences[i] != sequences[j])
                    dist_matrix[i, j] = d
                    dist_matrix[j, i] = d
        
        if n_clusters is None:
            n_clusters = max(2, self.basin_info.get('num_basins', 3) if self.basin_info else 3)
        
        condensed = squareform(dist_matrix)
        Z = linkage(condensed, method='average')
        self.cluster_labels = fcluster(Z, n_clusters, criterion='maxclust')
        
        return self.cluster_labels

print("TDA Basin Analyzer loaded!")


Ripser available for persistent homology
TDA Basin Analyzer loaded!


In [5]:
# Cell 4: Diversity-Aware Selection
class DiversitySelector:
    """Diversity-aware seed selection combining TDA clustering with quality ranking"""
    
    def __init__(self, tda_weight: float = 0.5, energy_weight: float = 0.5):
        self.tda_weight = tda_weight
        self.energy_weight = energy_weight
    
    def _normalize(self, values: List[float]) -> np.ndarray:
        """Normalize values to [0, 1] range"""
        values = np.array(values, dtype=float)
        min_val, max_val = values.min(), values.max()
        if max_val - min_val < 1e-10:
            return np.zeros_like(values)
        return (values - min_val) / (max_val - min_val)
    
    def select_seeds(self,
                     sequences: List[np.ndarray],
                     energies: List[int],
                     cluster_labels: np.ndarray,
                     n_seeds: int = 10,
                     diversity_bonus: float = 0.3) -> List[int]:
        """Select diverse, high-quality seeds for MTS"""
        n_samples = len(sequences)
        energy_scores = self._normalize(energies)
        
        # Greedy selection with diversity bonus
        selected = []
        selected_clusters = {}
        
        for _ in range(min(n_seeds, n_samples)):
            best_idx = -1
            best_score = float('inf')
            
            for i in range(n_samples):
                if i in selected:
                    continue
                
                # Diversity penalty for over-represented clusters
                cluster = cluster_labels[i]
                cluster_selected = selected_clusters.get(cluster, 0)
                diversity_penalty = diversity_bonus * cluster_selected
                
                adjusted_score = energy_scores[i] + diversity_penalty
                
                if adjusted_score < best_score:
                    best_score = adjusted_score
                    best_idx = i
            
            if best_idx >= 0:
                selected.append(best_idx)
                cluster = cluster_labels[best_idx]
                selected_clusters[cluster] = selected_clusters.get(cluster, 0) + 1
        
        return selected

print("Diversity Selector loaded!")


Diversity Selector loaded!


In [6]:
# Cell 5: Memetic Tabu Search
import random
from collections import deque

class MemeticTabuSearch:
    """Memetic Tabu Search optimizer for LABS"""
    
    def __init__(self, N: int, tabu_tenure: int = None,
                 population_size: int = 10, max_iterations: int = 5000,
                 stagnation_limit: int = 100):
        self.N = N
        self.tabu_tenure = tabu_tenure or N // 2
        self.population_size = population_size
        self.max_iterations = max_iterations
        self.stagnation_limit = stagnation_limit
        
        self.iteration_count = 0
        self.eval_count = 0
        self.best_energy = float('inf')
        self.best_sequence = None
        self.convergence_history = []
    
    def _labs_energy(self, sequence: np.ndarray) -> int:
        """Compute LABS energy (with counting)"""
        self.eval_count += 1
        N = len(sequence)
        energy = 0
        for k in range(1, N):
            C_k = sum(sequence[i] * sequence[i + k] for i in range(N - k))
            energy += C_k ** 2
        return energy
    
    def local_search(self, sequence: np.ndarray, max_steps: int = 100) -> Tuple[np.ndarray, int, int]:
        """Tabu search local optimization"""
        current = sequence.copy()
        current_energy = self._labs_energy(current)
        
        tabu_list = deque(maxlen=self.tabu_tenure)
        best_local = current.copy()
        best_local_energy = current_energy
        steps_without_improvement = 0
        
        for step in range(max_steps):
            best_move = -1
            best_move_energy = float('inf')
            
            for i in range(self.N):
                # Evaluate flip
                current[i] *= -1
                new_energy = self._labs_energy(current)
                current[i] *= -1
                
                # Check if move is allowed (not tabu or aspiration)
                if i not in tabu_list or new_energy < best_local_energy:
                    if new_energy < best_move_energy:
                        best_move = i
                        best_move_energy = new_energy
            
            if best_move < 0:
                break
            
            # Make the move
            current[best_move] *= -1
            current_energy = best_move_energy
            tabu_list.append(best_move)
            
            if current_energy < best_local_energy:
                best_local = current.copy()
                best_local_energy = current_energy
                steps_without_improvement = 0
            else:
                steps_without_improvement += 1
            
            if steps_without_improvement >= self.stagnation_limit // 2:
                break
        
        return best_local, best_local_energy, step + 1
    
    def crossover(self, parent1: np.ndarray, parent2: np.ndarray) -> np.ndarray:
        """Two-point crossover"""
        N = len(parent1)
        pt1, pt2 = sorted(random.sample(range(N), 2))
        child = parent1.copy()
        child[pt1:pt2] = parent2[pt1:pt2]
        return child
    
    def mutate(self, sequence: np.ndarray, rate: float = 0.1) -> np.ndarray:
        """Random bit flip mutation"""
        mutated = sequence.copy()
        for i in range(len(mutated)):
            if random.random() < rate:
                mutated[i] *= -1
        return mutated
    
    def optimize(self, initial_population: List[np.ndarray] = None,
                 verbose: bool = True) -> Tuple[np.ndarray, int, Dict]:
        """Run the full MTS optimization"""
        
        # Initialize population
        if initial_population is None:
            population = [np.random.choice([-1, 1], self.N) 
                         for _ in range(self.population_size)]
        else:
            population = [seq.copy() for seq in initial_population[:self.population_size]]
            while len(population) < self.population_size:
                population.append(np.random.choice([-1, 1], self.N))
        
        # Evaluate initial population
        pop_with_energy = [(seq, self._labs_energy(seq)) for seq in population]
        pop_with_energy.sort(key=lambda x: x[1])
        
        self.best_sequence = pop_with_energy[0][0].copy()
        self.best_energy = pop_with_energy[0][1]
        
        stagnation_counter = 0
        
        for iteration in range(self.max_iterations):
            self.iteration_count = iteration
            
            # Local search on each individual
            improved_pop = []
            for seq, energy in pop_with_energy:
                improved_seq, improved_energy, _ = self.local_search(seq, max_steps=50)
                improved_pop.append((improved_seq, improved_energy))
            
            # Update best
            improved_pop.sort(key=lambda x: x[1])
            if improved_pop[0][1] < self.best_energy:
                self.best_energy = improved_pop[0][1]
                self.best_sequence = improved_pop[0][0].copy()
                stagnation_counter = 0
                
                if verbose:
                    merit = self.N ** 2 / (2 * self.best_energy) if self.best_energy > 0 else float('inf')
                    print(f"Iter {iteration}: New best E={self.best_energy}, F={merit:.3f}")
            else:
                stagnation_counter += 1
            
            self.convergence_history.append(self.best_energy)
            
            if stagnation_counter >= self.stagnation_limit:
                if verbose:
                    print(f"Converged at iteration {iteration}")
                break
            
            # Create new population via crossover/mutation
            new_population = [improved_pop[0]]  # Elitism
            
            while len(new_population) < self.population_size:
                p1 = min(random.sample(improved_pop, 2), key=lambda x: x[1])[0]
                p2 = min(random.sample(improved_pop, 2), key=lambda x: x[1])[0]
                
                child = self.crossover(p1, p2)
                mutation_rate = 0.05 + 0.15 * (stagnation_counter / self.stagnation_limit)
                child = self.mutate(child, mutation_rate)
                
                child_energy = self._labs_energy(child)
                new_population.append((child, child_energy))
            
            pop_with_energy = new_population
        
        return self.best_sequence, self.best_energy, {
            'iterations': self.iteration_count,
            'evaluations': self.eval_count,
            'convergence_history': self.convergence_history
        }

print("Memetic Tabu Search loaded!")


Memetic Tabu Search loaded!


In [7]:
# Cell 6: Complete Pipeline
import time

class LABSHybridPipeline:
    """Full TDA filtering pipeline for quantum-enhanced LABS optimization"""
    
    def __init__(self, N: int, dcqo_layers: int = 3, dcqo_samples: int = 2000,
                 mts_population: int = 10, mts_iterations: int = 5000):
        self.N = N
        self.dcqo_samples = dcqo_samples
        
        self.dcqo = DCQOSampler(N=N, num_layers=dcqo_layers)
        self.tda = TDABasinAnalyzer(persistence_threshold=0.15)
        self.selector = DiversitySelector()
        self.mts = MemeticTabuSearch(N=N, population_size=mts_population,
                                     max_iterations=mts_iterations)
        
        self.results = {}
        self.timing = {}
    
    def run(self, verbose: bool = True) -> Dict:
        """Execute the full pipeline"""
        total_start = time.time()
        
        # ========== STAGE 0: DCQO Sampling ==========
        if verbose:
            print(f"\n{'='*50}")
            print(f"LABS Hybrid Pipeline - N={self.N}")
            print(f"{'='*50}")
            print("\n[Stage 0] DCQO Quantum Sampling...")
        
        stage_start = time.time()
        dcqo_results = self.dcqo.sample(num_samples=self.dcqo_samples)
        
        # Deduplicate
        seen = set()
        unique_samples = []
        for seq, energy in dcqo_results:
            key = tuple(seq)
            if key not in seen:
                seen.add(key)
                unique_samples.append((seq, energy))
        
        sequences = [s[0] for s in unique_samples]
        energies = [s[1] for s in unique_samples]
        
        self.timing['dcqo'] = time.time() - stage_start
        
        if verbose:
            print(f"  Generated {len(sequences)} unique samples")
            print(f"  Best DCQO energy: {min(energies)}")
            print(f"  Time: {self.timing['dcqo']:.2f}s")
        
        # ========== STAGE 1: TDA Basin Analysis ==========
        if verbose:
            print("\n[Stage 1] TDA Basin Analysis...")
        
        stage_start = time.time()
        tda_results = self.tda.compute_persistence(sequences, energies)
        cluster_labels = self.tda.assign_clusters(sequences, tda_results['distance_matrix'])
        
        self.timing['tda'] = time.time() - stage_start
        
        if verbose:
            n_clusters = len(set(cluster_labels))
            print(f"  Identified {n_clusters} basins")
            print(f"  Median persistence: {tda_results['basin_info'].get('median_persistence', 0):.2f}")
            print(f"  Time: {self.timing['tda']:.2f}s")
        
        # ========== STAGE 2: Diverse Seed Selection ==========
        if verbose:
            print("\n[Stage 2] Diversity-Aware Selection...")
        
        stage_start = time.time()
        
        selected_indices = self.selector.select_seeds(
            sequences=sequences,
            energies=energies,
            cluster_labels=cluster_labels,
            n_seeds=self.mts.population_size,
            diversity_bonus=0.3
        )
        
        selected_sequences = [sequences[i] for i in selected_indices]
        selected_energies = [energies[i] for i in selected_indices]
        selected_clusters = [cluster_labels[i] for i in selected_indices]
        
        self.timing['selection'] = time.time() - stage_start
        
        if verbose:
            print(f"  Selected {len(selected_sequences)} seeds")
            print(f"  From {len(set(selected_clusters))} different clusters")
            print(f"  Energy range: [{min(selected_energies)}, {max(selected_energies)}]")
            print(f"  Time: {self.timing['selection']:.2f}s")
        
        # ========== STAGE 3: MTS Optimization ==========
        if verbose:
            print("\n[Stage 3] Memetic Tabu Search...")
        
        stage_start = time.time()
        
        best_sequence, best_energy, mts_stats = self.mts.optimize(
            initial_population=selected_sequences,
            verbose=verbose
        )
        
        self.timing['mts'] = time.time() - stage_start
        self.timing['total'] = time.time() - total_start
        
        # ========== Results Summary ==========
        best_merit = merit_factor(best_sequence)
        
        self.results = {
            'N': self.N,
            'best_sequence': best_sequence.tolist(),
            'best_energy': int(best_energy),
            'merit_factor': float(best_merit),
            'dcqo_samples': len(sequences),
            'dcqo_best_energy': int(min(energies)),
            'n_basins': len(set(cluster_labels)),
            'mts_iterations': mts_stats['iterations'],
            'mts_evaluations': mts_stats['evaluations'],
            'timing': self.timing
        }
        
        if verbose:
            print(f"\n{'='*50}")
            print("RESULTS SUMMARY")
            print(f"{'='*50}")
            print(f"Best Energy: {best_energy}")
            print(f"Merit Factor: {best_merit:.4f}")
            print(f"MTS Iterations: {mts_stats['iterations']}")
            print(f"Total Time: {self.timing['total']:.2f}s")
            print(f"\nTiming Breakdown:")
            for stage, t in self.timing.items():
                if stage != 'total':
                    pct = 100 * t / self.timing['total']
                    print(f"  {stage}: {t:.2f}s ({pct:.1f}%)")
        
        return self.results

print("Pipeline loaded! Ready to run.")


Pipeline loaded! Ready to run.


In [8]:
# Cell 7: RUN THE PIPELINE
# Start with a small N for testing, then scale up

# Small test (fast, ~30 seconds)
print("="*60)
print("SMALL TEST RUN (N=12)")
print("="*60)

pipeline_small = LABSHybridPipeline(
    N=12,
    dcqo_layers=2,
    dcqo_samples=500,
    mts_population=8,
    mts_iterations=500
)

results_small = pipeline_small.run(verbose=True)


SMALL TEST RUN (N=12)

LABS Hybrid Pipeline - N=12

[Stage 0] DCQO Quantum Sampling...
  Generated 434 unique samples
  Best DCQO energy: 10
  Time: 0.13s

[Stage 1] TDA Basin Analysis...
  Identified 157 basins
  Median persistence: 1.07
  Time: 0.44s

[Stage 2] Diversity-Aware Selection...
  Selected 8 seeds
  From 8 different clusters
  Energy range: [10, 22]
  Time: 0.00s

[Stage 3] Memetic Tabu Search...
Converged at iteration 99

RESULTS SUMMARY
Best Energy: 10
Merit Factor: 7.2000
MTS Iterations: 99
Total Time: 10.49s

Timing Breakdown:
  dcqo: 0.13s (1.3%)
  tda: 0.44s (4.2%)
  selection: 0.00s (0.0%)
  mts: 9.91s (94.5%)


In [9]:
# Cell 8: FULL SCALE RUN
# This will use more compute - adjust based on your credits

print("\n" + "="*60)
print("FULL SCALE RUN (N=20)")
print("="*60)

pipeline_full = LABSHybridPipeline(
    N=20,
    dcqo_layers=3,
    dcqo_samples=2000,
    mts_population=10,
    mts_iterations=2000
)

results_full = pipeline_full.run(verbose=True)

# Save results
import json
with open('labs_results_n20.json', 'w') as f:
    json.dump(results_full, f, indent=2)
print("\nResults saved to labs_results_n20.json")



FULL SCALE RUN (N=20)

LABS Hybrid Pipeline - N=20

[Stage 0] DCQO Quantum Sampling...
  Generated 1804 unique samples
  Best DCQO energy: 58
  Time: 176.53s

[Stage 1] TDA Basin Analysis...
  Identified 879 basins
  Median persistence: 1.33
  Time: 7.21s

[Stage 2] Diversity-Aware Selection...
  Selected 10 seeds
  From 10 different clusters
  Energy range: [58, 74]
  Time: 0.01s

[Stage 3] Memetic Tabu Search...
Iter 0: New best E=34, F=5.882
Iter 1: New best E=26, F=7.692
Converged at iteration 101

RESULTS SUMMARY
Best Energy: 26
Merit Factor: 7.6923
MTS Iterations: 101
Total Time: 235.22s

Timing Breakdown:
  dcqo: 176.53s (75.0%)
  tda: 7.21s (3.1%)
  selection: 0.01s (0.0%)
  mts: 51.47s (21.9%)

Results saved to labs_results_n20.json


In [ ]:
# Cell 8: FULL SCALE RUN
# This will use more compute - adjust based on your credits

print("\n" + "="*60)
print("FULL SCALE RUN (N=48)")
print("="*60)

pipeline_full = LABSHybridPipeline(
    N=48,
    dcqo_layers=3,
    dcqo_samples=2000,
    mts_population=10,
    mts_iterations=2000
)

results_full = pipeline_full.run(verbose=True)

# Save results
import json
with open('labs_results_n20.json', 'w') as f:
    json.dump(results_full, f, indent=2)
print("\nResults saved to labs_results_n20.json")

In [13]:
# Run this in a cell before your pipeline
from unittest.mock import patch
import qiskit_aer

original_run = qiskit_aer.AerSimulator.run

def debug_run(self, circuits, **kwargs):
    print(f"AerSimulator.run called with method={self._method}, circuits={type(circuits)}")
    if hasattr(circuits, '__len__'):
        print(f"  Number of circuits: {len(circuits)}")
    import traceback
    traceback.print_stack(limit=10)
    return original_run(self, circuits, **kwargs)

qiskit_aer.AerSimulator.run = debug_run


In [14]:
# Cell 8: FULL SCALE RUN
# This will use more compute - adjust based on your credits

print("\n" + "="*60)
print("FULL SCALE RUN (N=48)")
print("="*60)

pipeline_full = LABSHybridPipeline(
    N=48,
    dcqo_layers=3,
    dcqo_samples=2000,
    mts_population=10,
    mts_iterations=2000
)

results_full = pipeline_full.run(verbose=True)

# Save results
import json
with open('labs_results_n20.json', 'w') as f:
    json.dump(results_full, f, indent=2)
print("\nResults saved to labs_results_n20.json")


FULL SCALE RUN (N=48)

LABS Hybrid Pipeline - N=48

[Stage 0] DCQO Quantum Sampling...


AttributeError: 'AerSimulator' object has no attribute '_method'